In [1]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pickle
import re
import csv

In [16]:
URL = 'https://www.walmart.com/account/giftcards/balance'

In [22]:
option = webdriver.ChromeOptions()
option.headless=False
driver = uc.Chrome(options=option)

In [23]:
driver.get(URL)

In [20]:
wait = WebDriverWait(driver, 10)
with open('numbers.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        number = row[0]
        pin = row[1]
        
        # Run this line if the gift card numbers in the CSV file do not include a space after every 4th letter. 
        #number = number[:4]+ ' ' + number[4:8]+' '+number[8:12]+' '+number[12:]
        
        try:
            wait.until(EC.presence_of_element_located((By.ID, 'react-aria-1')))
            driver.find_element(By.ID, 'react-aria-1').send_keys(number)
            wait.until(EC.text_to_be_present_in_element_value((By.ID, 'react-aria-1'), number))
            driver.find_element(By.ID, 'react-aria-2').send_keys(pin)
            wait.until(EC.text_to_be_present_in_element_value((By.ID, 'react-aria-2'), pin))
            driver.find_element(By.ID, 'react-aria-2').submit()
            if wait.until(EC.text_to_be_present_in_element((By.ID, "ld_modalTitle_0"), 'Gift card balance')):

                html = driver.page_source
                start = '<span class="balance ">'
                end = '</span>'
                result = re.search('%s(.*)%s' %(start, end), html).group(1)

                val = {'number': number, 'balance': result}
                with open('balance.csv', 'a') as csvfile:
                    fieldnames = ['number', 'balance']
                    writer = csv.DictWriter(csvfile,fieldnames=fieldnames)
                    writer.writerow(val)

            driver.refresh()
        except AttributeError:
            print('Could not retrieve: '+number)
driver.quit()